In [1]:
# Import dependencies
import pandas as pd
import gmaps
import requests
# Import API key
from config import g_key

In [2]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
# Store CSV from previous section into DF
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Bredasdorp,ZA,2021-01-31 19:42:52,-34.5322,20.0403,66.00,93,100,20.00
1,1,Virginia Beach,US,2021-01-31 19:39:02,36.8529,-75.9780,43.00,98,100,1.01
2,2,Kutum,SD,2021-01-31 19:42:52,14.2000,24.6667,61.63,28,0,7.09
3,3,Bluff,NZ,2021-01-31 19:42:52,-46.6000,168.3333,57.36,78,56,5.37
4,4,San Quintin,MX,2021-01-31 19:37:58,30.4833,-115.9500,62.67,51,98,3.78


In [4]:
# Configure gmaps to use Google API key
gmaps.configure(api_key=g_key)

In [5]:
# High Temp heatmap
# Get Lat-lon
locations = city_data_df[["Lat","Lng"]]
# Get high temp
max_temp = city_data_df ["Max Temp"]
# Assign fig variable
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign heatmap variable
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False,
                                max_intensity=300,point_radius=4)
# Add heatmap layer
fig.add_layer(heat_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# % Humidity heatmap
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300,
                                point_radius=4)
fig.add_layer(heat_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# % Cloudiness heatmap
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300,
                                point_radius=4)
fig.add_layer(heat_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Wind speed heatmap
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=wind, dissipating=False, max_intensity=300,
                                point_radius=4)
fig.add_layer(heat_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
# Ask customer for min/max temps
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [10]:
# Filter DF for cities that meet temperature criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)].dropna()
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
6,6,Vaini,TO,2021-01-31 19:42:52,-21.2000,-175.2000,77.00,100,90,14.97
14,14,Rikitea,PF,2021-01-31 19:42:53,-23.1203,-134.9692,78.93,64,16,9.13
17,17,Grand-Santi,GF,2021-01-31 19:42:53,4.2500,-54.3833,79.72,91,15,4.92
20,20,Georgetown,MY,2021-01-31 19:42:54,5.4112,100.3354,78.80,78,20,4.61
32,32,Carnarvon,AU,2021-01-31 19:42:54,-24.8667,113.6333,86.00,48,74,16.11
36,36,Hambantota,LK,2021-01-31 19:42:55,6.1241,81.1185,75.20,100,75,5.75
40,40,Labuhan,ID,2021-01-31 19:42:55,-6.8844,112.2051,77.77,82,100,14.27
41,41,Saint-Pierre,RE,2021-01-31 19:42:55,-21.3393,55.4781,82.40,65,0,18.41
42,42,Butaritari,KI,2021-01-31 19:42:55,3.0707,172.7902,80.62,83,36,13.13
45,45,Kontagora,NG,2021-01-31 19:42:55,10.3999,5.4695,81.18,16,100,4.99


In [11]:
preferred_cities_df.count()

City_ID       174
City          174
Country       174
Date          174
Lat           174
Lng           174
Max Temp      174
Humidity      174
Cloudiness    174
Wind Speed    174
dtype: int64

In [12]:
# Create DF to store hotel names
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Vaini,TO,77.00,-21.2000,-175.2000,
14,Rikitea,PF,78.93,-23.1203,-134.9692,
17,Grand-Santi,GF,79.72,4.2500,-54.3833,
20,Georgetown,MY,78.80,5.4112,100.3354,
32,Carnarvon,AU,86.00,-24.8667,113.6333,
36,Hambantota,LK,75.20,6.1241,81.1185,
40,Labuhan,ID,77.77,-6.8844,112.2051,
41,Saint-Pierre,RE,82.40,-21.3393,55.4781,
42,Butaritari,KI,80.62,3.0707,172.7902,
45,Kontagora,NG,81.18,10.3999,5.4695,


In [13]:
# Set parameters for API hotel search
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [15]:
# Iterate through DF
for index, row in hotel_df.iterrows():
    # Get the lat/lon
    lat = row["Lat"]
    lng = row["Lng"]

    # Add lat/lon location key for params dictionary
    params["location"] = f"{lat},{lng}"

    # Use search term: "lodging" and our lat/lon
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get JSON data from search
    hotels = requests.get(base_url, params=params).json()
    # Grab first hotel from results and store name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... Skipping")

Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping
Hotel not found... Skipping


In [16]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
6,Vaini,TO,77.00,-21.2000,-175.2000,Keleti Beach Resort
14,Rikitea,PF,78.93,-23.1203,-134.9692,Pension Maro'i
17,Grand-Santi,GF,79.72,4.2500,-54.3833,Hotel La Villa Couacou
20,Georgetown,MY,78.80,5.4112,100.3354,Cititel Penang
32,Carnarvon,AU,86.00,-24.8667,113.6333,Hospitality Carnarvon
36,Hambantota,LK,75.20,6.1241,81.1185,Bungalow 63
40,Labuhan,ID,77.77,-6.8844,112.2051,PT PPI Lamongan Tambak C
41,Saint-Pierre,RE,82.40,-21.3393,55.4781,Lindsey Hôtel
42,Butaritari,KI,80.62,3.0707,172.7902,Isles Sunset Lodge
45,Kontagora,NG,81.18,10.3999,5.4695,City Gate Inn


In [22]:
# Add temperature heatmap for hotel DF
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp,0) for temp in max_temp], dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call figure
fig

Figure(layout=FigureLayout(height='420px'))

In [20]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [21]:
# Store DF row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]